In [1]:
import pandas as pd

In [3]:
messages=pd.read_csv('C:\\Users\\KEERTI\\Downloads\\spam_classifier-NLP--main\\spam_classifier-NLP--main\\SMSSpamCollection', sep='\t',names=["label", "text"])

In [4]:
import re
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KEERTI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [6]:
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [15]:
y=pd.get_dummies(messages['label'])

In [16]:
y.head()

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [17]:
y=y.iloc[:,1].values

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [10]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

In [11]:
y_pred=spam_detect_model.predict(X_test)

In [12]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[946,   9],
       [  7, 153]], dtype=int64)

In [13]:
from sklearn.metrics import accuracy_score
ac=accuracy_score(y_test,y_pred)
ac

0.9856502242152466

In [32]:
def classify_sentence(sentence):
    # Preprocess the input sentence
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    sentence = sentence.lower()
    sentence = sentence.split()
    sentence = [ps.stem(word) for word in sentence if not word in stopwords.words('english')]
    sentence = ' '.join(sentence)

    # Transform the input sentence into a 2D array-like format
    sentence_vectorized = cv.transform([sentence]).toarray()

    # Make a prediction
    prediction = spam_detect_model.predict(sentence_vectorized)

    # Return 1 for spam and 0 for not spam
    return prediction[0]

# Example usage:
input_sentence = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
result = classify_sentence(input_sentence)
if result == 1:
    print("This sentence is spam.")
else:
    print("This sentence is not spam.")

This sentence is spam.
